## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [4]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32,3,3,input_shape=(32,32,3),activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
#classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(16,3,3,activation='relu'))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(units=100,activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=11)

Epoch 1/11
50000/50000 [==============================] - 88s 2ms/step - loss: 1.5980 - accuracy: 0.4897
Epoch 2/11
50000/50000 [==============================] - 84s 2ms/step - loss: 0.9374 - accuracy: 0.6713
Epoch 3/11
50000/50000 [==============================] - 81s 2ms/step - loss: 0.6494 - accuracy: 0.7723
Epoch 4/11
50000/50000 [==============================] - 79s 2ms/step - loss: 0.4195 - accuracy: 0.8559
Epoch 5/11
50000/50000 [==============================] - 80s 2ms/step - loss: 0.2623 - accuracy: 0.9104
Epoch 6/11
50000/50000 [==============================] - 81s 2ms/step - loss: 0.1843 - accuracy: 0.9368
Epoch 7/11
50000/50000 [==============================] - 83s 2ms/step - loss: 0.1605 - accuracy: 0.9452
Epoch 8/11
50000/50000 [==============================] - 84s 2ms/step - loss: 0.1376 - accuracy: 0.9541
Epoch 9/11
50000/50000 [==============================] - 82s 2ms/step - loss: 0.1143 - accuracy: 0.9619
Epoch 10/11
50000/50000 [==============================

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[7.0586007e-05, 3.2136005e-09, 8.8861422e-04, 9.8367625e-01,
        1.2084812e-02, 3.1456461e-03, 9.2004819e-05, 4.1875141e-05,
        1.4845013e-07, 6.9110811e-08]], dtype=float32)